```{contents}
```
## Map–Reduce 

**Map–Reduce** in LangGraph is a **parallel, state-driven execution pattern** for decomposing a large problem into independent subtasks (**Map**), processing them concurrently, and then combining their outputs into a final result (**Reduce**).
LangGraph implements this pattern using **fan-out, fan-in, and state aggregation** within a persistent execution graph.

---

### **1. Why Map–Reduce is Needed in LLM Systems**

Large tasks often exceed a single LLM call’s capacity:

| Problem Type          | Why Map–Reduce Helps            |
| --------------------- | ------------------------------- |
| Large documents       | Chunk + summarize in parallel   |
| Multi-source research | Query many sources concurrently |
| Batch data processing | Scale throughput                |
| Long reasoning chains | Divide cognitive load           |
| Multi-agent workloads | Independent agent execution     |

---

### **2. Conceptual Structure**

```
                ┌── Map A ──┐
Input → Split → ├── Map B ──┤ → Reduce → Final Output
                └── Map C ──┘
```

LangGraph models this as:

> **Fan-Out → Parallel Nodes → Fan-In → Aggregation**

---

### **3. State Design**

The shared state holds both the **work items** and the **partial results**.

```python
class State(TypedDict):
    tasks: list[str]
    partial_results: list[str]
    final: str
```

Reducers combine partial outputs safely.

```python
def reduce_results(old, new):
    return old + new
```

---

### **4. Implementation in LangGraph**

### **Step 1 — Map Node**

```python
def map_worker(state):
    task = state["tasks"].pop(0)
    result = task.upper()  # simulate processing
    return {"partial_results": [result]}
```

### **Step 2 — Reduce Node**

```python
def reduce_node(state):
    combined = "\n".join(state["partial_results"])
    return {"final": combined}
```

---

### **5. Graph Construction**

```python
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("map", map_worker)
builder.add_node("reduce", reduce_node)

builder.set_entry_point("map")

# loop until all tasks are processed
def router(state):
    if state["tasks"]:
        return "map"
    return "reduce"

builder.add_conditional_edges("map", router, {
    "map": "map",
    "reduce": "reduce"
})

builder.add_edge("reduce", END)

graph = builder.compile()
```

---

### **6. Execution**

```python
input_state = {
    "tasks": ["alpha", "beta", "gamma"],
    "partial_results": []
}

output = graph.invoke(input_state)
print(output["final"])
```

**Output**

```
ALPHA
BETA
GAMMA
```

---

### **7. Parallel Map Variant (High-Throughput)**

```python
builder.add_node("map", map_worker, retry=2)
builder.add_node("map_parallel", map_worker)
builder.add_edge("split", "map_parallel")
builder.add_edge("map_parallel", "join")
```

LangGraph can schedule independent nodes **concurrently** using async execution.

---

### **8. Common LangGraph Map–Reduce Use Cases**

| Use Case               | Pattern                           |
| ---------------------- | --------------------------------- |
| Document summarization | Chunk → summarize → combine       |
| Research aggregation   | Query sources → merge             |
| Multi-agent analysis   | Agents analyze → consensus        |
| Data pipelines         | Partition → transform → aggregate |
| Evaluation harness     | Test cases → score → report       |

---

### **9. Production Considerations**

| Concern         | Solution                   |
| --------------- | -------------------------- |
| Fault tolerance | Retries, checkpoints       |
| Scalability     | Async + batching           |
| Memory safety   | Reducers + partial updates |
| Cost control    | Parallel token budgeting   |
| Observability   | Tracing + metrics          |

---

### **10. Mental Model**

LangGraph Map–Reduce behaves like a **distributed cognitive system**:

> **Decompose → Think in Parallel → Integrate Knowledge**

This allows LLM applications to scale both **computation** and **reasoning capacity** beyond single-model limitations.


### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Define shared state
# -----------------------------
class State(TypedDict):
    tasks: List[str]
    partial_results: List[str]
    final: str

# -----------------------------
# 2. Map worker
# -----------------------------
def map_worker(state: State):
    task = state["tasks"].pop(0)
    result = f"Processed({task})"
    return {"partial_results": [result]}

# -----------------------------
# 3. Reduce node
# -----------------------------
def reduce_node(state: State):
    combined = " | ".join(state["partial_results"])
    return {"final": combined}

# -----------------------------
# 4. Router for looping
# -----------------------------
def router(state: State):
    if state["tasks"]:
        return "map"
    return "reduce"

# -----------------------------
# 5. Build the graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("map", map_worker)
builder.add_node("reduce", reduce_node)

builder.set_entry_point("map")

builder.add_conditional_edges(
    "map",
    router,
    {"map": "map", "reduce": "reduce"}
)

builder.add_edge("reduce", END)

graph = builder.compile()

# -----------------------------
# 6. Execute
# -----------------------------
result = graph.invoke({
    "tasks": ["task1", "task2", "task3"],
    "partial_results": []
})

print("Final Output:", result["final"])


Final Output: Processed(task3)
